In [10]:
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI 
import os
import sys
from typing import Dict, Any, List, Optional
import logging
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('agent.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)


In [12]:

class AzureOpenAIConfig:
    """Production Azure OpenAI configuration with validation and health checks."""
    
    def __init__(self):
        """Initialize and validate Azure OpenAI configuration."""
        # Load configuration from environment
        self.api_key = os.getenv("AZURE_OPENAI_API_KEY")
        self.endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        self.api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
        self.deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4.1")
        self.gpt4_deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT_NAME", "gpt-4.1")
        
        # Configuration limits
        self.max_retries = int(os.getenv("MAX_RETRIES", "3"))
        self.rate_limit_calls = int(os.getenv("RATE_LIMIT_CALLS_PER_MINUTE", "60"))
        self.max_conversation_messages = int(os.getenv("MAX_CONVERSATION_MESSAGES", "100"))
        self.request_timeout = int(os.getenv("REQUEST_TIMEOUT", "60"))
        
        # Validate configuration
        self._validate()
        
    def _validate(self):
        """Validate configuration and fail fast if invalid."""
        errors = []
        
        if not self.api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
        elif not self.endpoint.startswith("https://"):
            errors.append("AZURE_OPENAI_ENDPOINT must start with https://")
        
        if not self.deployment_name:
            errors.append("AZURE_OPENAI_DEPLOYMENT_NAME is required")
        
        if errors:
            error_msg = "Configuration validation failed:\n" + "\n".join(f"  - {e}" for e in errors)
            logger.error(error_msg)
            raise ValueError(error_msg)
        
        # Ensure endpoint has trailing slash
        if not self.endpoint.endswith("/"):
            self.endpoint += "/"
        
        logger.info("Azure OpenAI configuration validated successfully")
    
    def get_model(
        self,
        deployment_name: Optional[str] = None,
        temperature: float = 0.7,
        max_tokens: Optional[int] = None,
    ) -> AzureChatOpenAI:
        """Get configured Azure OpenAI model instance."""
        deployment = deployment_name or self.deployment_name
        
        return AzureChatOpenAI(
            azure_deployment=deployment,
            api_version=self.api_version,
            temperature=temperature,
            max_tokens=max_tokens,
            azure_endpoint=self.endpoint,
            api_key=self.api_key,
            timeout=self.request_timeout,
        )
    
    def health_check(self) -> Dict[str, Any]:
        """Perform health check on Azure OpenAI connection."""
        try:
            model = self.get_model()
            response = model.invoke([HumanMessage(content="Health check")])
            
            return {
                "status": "healthy",
                "endpoint": self.endpoint,
                "deployment": self.deployment_name,
                "timestamp": datetime.utcnow().isoformat(),
            }
        except Exception as e:
            logger.error(f"Health check failed: {e}")
            return {
                "status": "unhealthy",
                "error": str(e),
                "timestamp": datetime.utcnow().isoformat(),
            }

In [13]:
client = AzureOpenAIConfig()


2026-02-15 21:28:52,312 - __main__ - INFO - Azure OpenAI configuration validated successfully
